In [6]:
import torch
import numpy as np
import output

# Initialize spins and interaction matrix
def initialize_spins_and_interactions(n, seed):
    torch.manual_seed(seed)
    spins = torch.randint(0, 2, (n,)) * 2 - 1  # {1, -1}
    interactions = torch.normal(0.0, 1.0, (n, n))
    return spins, interactions

# Calculate energy
def calculate_energy(spins, h, interactions):
    field_term = h * torch.sum(spins)  # h is a scalar
    interaction_term = torch.sum(interactions * torch.outer(spins, spins)) - torch.sum(torch.diagonal(interactions))
    return -(field_term + interaction_term)

# Simulated annealing simulation
def simulated_annealing(n, h, T, seed, iterations):
    torch.manual_seed(seed)
    spins, interactions = initialize_spins_and_interactions(n, seed)
    best_energy = 0

    for _ in range(iterations):
        current_spins = spins.clone()

        # Flip a random spin
        flip_index = torch.randint(0, n, (1,)).item()
        spins[flip_index] *= -1

        # Energy difference and acceptance
        delta_E = calculate_energy(spins, h, interactions) - calculate_energy(current_spins, h, interactions)
        if delta_E < 0 or torch.rand(1).item() < torch.exp(-delta_E / T):
            T *= 0.99999
            continue  # Accept new state
        else:
            T *= 0.99999
            spins = current_spins  # Revert to original state
        
        if calculate_energy(spins, h, interactions) < best_energy:
            best_energy = calculate_energy(spins, h, interactions)
        
        # Annealing


    return calculate_energy(spins, h, interactions), best_energy

# Parameters
n = 100
h = 0.001  # Magnetic field scalar
T = 10000
seed = 42
iterations = 1000000
final_list = []
best_list = []

# Calculate mean energy
for _ in range(100):
    T = 10000
    # Caluculate energy
    final_energy, best_energy = simulated_annealing(n, h, T, seed, iterations)
    final_energy = final_energy.item()
    best_energy = best_energy.item()

    # Append energy
    final_list.append(final_energy)
    best_list.append(best_energy)

    # Change seed
    seed += 1

result = [[f, b] for f, b in zip(final_list, best_list)]
result.insert(0, ['final', 'best'])
data = output.matrix_to_csv(f'T{T}_seed{seed}', result)

print("Final mean energy", np.mean(final_list))
print('Best mean energy', np.mean(best_list))

Final mean energy -1027.1172845458984
Best mean energy -1028.8479608154296


In [ ]:
#simulated annealing
import torch

# set J
def initialize_random_parameters(n, seed1):
    torch.manual_seed(seed1)
    x = torch.tensor([1 if torch.rand(1).item() < 0.5 else -1 for _ in range(n)])
    J = torch.normal(mean=0.0, std=1.0, size=(n, n))
    return x, J

# caluculate energy
def final_energy(x, h, J):
    magnetic_field = torch.sum(h * x)
    interaction = torch.sum(J * torch.outer(x, x)) - torch.sum(torch.diagonal(J * torch.outer(x, x)))
    
    return -(magnetic_field + interaction)

def delta_energy(current_spin, temp_spin, h, J):
    return final_energy(temp_spin, h, J) - final_energy(current_spin, h, J)

def accept_prob(current_spin, temp_spin, h, J, T):
    delta = delta_energy(current_spin, temp_spin, h, J)
    print(delta)

    if delta < 0:
        accept_prob = 1
    else:
        accept_prob = torch.exp(- delta / T)

    return accept_prob

def judge(current_spin, temp_spin, h, J, T, seed2):
    torch.manual_seed(seed2)
    
    if torch.rand(1) < accept_prob(current_spin, temp_spin, h, J, T):
        return True
    else:
        return False

def simulation(n, h, T, seed1, seed2, iter):
    
    x, J = initialize_random_parameters(n, seed1)
    
    for i in range(iter):
        current_spin = x.clone() # current spin

        print(final_energy(x, h, J))

        random_index = torch.randint(0, len(x), (1,)).item() # create random index
        x[random_index] *= -1

        if judge(current_spin, x, h, J, T, seed2) == False:
            x = current_spin

    return final_energy(x, h, J)

n = 10
h = 0.001
T = 1.0
seed1 = 42
seed2 = 42
iter = 3

print(simulation(n, h, T, seed1, seed2, iter).item())

